In [1]:
# Using Drive as storage and github for version controll.

from google.colab import drive, userdata
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Setup Paths (Change to your actual repo name)
REPO_PATH = "/content/drive/MyDrive/ML/DL_With_Pytorch"
%cd {REPO_PATH}

# 3. Secure Auth
token = userdata.get('GH_TOKEN')
username = "barada02"
repo = "DL_With_Pytorch"
!git remote set-url origin https://{token}@github.com/{username}/{repo}.git

# 4. Identity
!git config --global user.email "Chandanbarada2@gmail.com"
!git config --global user.name "Kumar"

!git pull origin main
print("✅ Environment Ready!")

Mounted at /content/drive
/content/drive/MyDrive/ML/DL_With_Pytorch
From https://github.com/barada02/DL_With_Pytorch
 * branch            main       -> FETCH_HEAD
Already up to date.
✅ Environment Ready!


In [18]:
# Push notebook changes to GitHub
# IMPORTANT: Press Ctrl+S (Save) before running this!
!git add .
!git commit -m "Transpose and Permute use"
!git push origin main

[main a62df99] Transpose and Permute use
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite 01_Tensor_03_Shapes.ipynb (80%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 480 bytes | 96.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/barada02/DL_With_Pytorch.git
   b35fb12..a62df99  main -> main


# Tensor Shapes

Manipulating tensor shapes is the bread and butter of deep learning. In PyTorch, these operations allow you to reorganize data to fit the expected input of different layers (like moving from a Convolutional layer to a Linear one)

In [6]:
import torch
torch.__version__


'2.9.0+cpu'

# The flatten
### Reshape and view

Both change the dimensions of a tensor without changing its data, but they handle memory differently.

* view(): The OG method. It is very fast because it creates a "view" of the original data without copying it. Requirement: The tensor must be contiguous in memory. If you’ve just transposed a tensor, view() might throw an error.

* reshape(): The more robust sibling. It tries to return a view if possible, but if the data isn't contiguous, it will silently copy the data to a new memory block.

Pro Tip: Use -1 as a dimension to let PyTorch automatically calculate the size for that slot based on the remaining dimensions.

In [8]:


# A batch of 4 images, 3 color channels, 28x28 pixels
x = torch.randn(4, 3, 28, 28)
print(f"Original shape: {x.shape}")

# Use -1 to say: "Keep the batch size (4), but squash everything else"
flattened = x.view(4, -1)
print(f"Flattened shape: {flattened.shape}") # [4, 2352]

Original shape: torch.Size([4, 3, 28, 28])
Flattened shape: torch.Size([4, 2352])


# Squeeze and Unsqueeze
These are used to add or remove "singleton" dimensions (dimensions of size 1).

* ```unsqueeze(dim)```: Adds a dimension of size 1 at the specified index.

  * Example: Changing a shape from [3, 224, 224] to [1, 3, 224, 224] to add a "batch" dimension.

* ```squeeze(dim)```: Removes a dimension of size 1. If no dimension is specified, it removes all dimensions of size 1.

   * Example: Changing [1, 10] to [10].

> **PyTorch models usually expect a Batch dimension. If you have a single image, you have to "fake" a batch of one.**

In [11]:
# A single RGB image
img = torch.randn(3, 224, 224)
print(f"Single image: {img.shape}")

# Add a batch dimension at index 0
batch_img = img.unsqueeze(0)
print(f"After unsqueeze(0): {batch_img.shape}") # [1, 3, 224, 224]

# Remove it back
back_to_img = batch_img.squeeze(0)
print(f"After squeeze(0): {back_to_img.shape}")

Single image: torch.Size([3, 224, 224])
After unsqueeze(0): torch.Size([1, 3, 224, 224])
After squeeze(0): torch.Size([3, 224, 224])


# 3. Transpose and Permute
These operations swap the axes of the tensor.

* transpose(dim0, dim1): Swaps exactly two dimensions. This is common in NLP or for simple matrix flips.

* permute(dims): A more powerful version of transpose that can reorder any number of dimensions at once.

  * Example: Converting an image from HWC (Height, Width, Channels) to CHW format: tensor.permute(2, 0, 1).

In [16]:
# Typical PyTorch tensor: [Channels, Height, Width]
img_tensor = torch.randn(3, 128, 128)

# Permute to [Height, Width, Channels] for plotting
img_for_plt = img_tensor.permute(1, 2, 0)
print(f"Permuted for plotting: {img_for_plt.shape}") # [128, 128, 3]

# Transpose is usually for just two specific dims
weight_matrix = torch.randn(10, 5)
transposed = weight_matrix.transpose(0, 1)
print(f"Transposed matrix: {transposed.shape}") # [5, 10]

Permuted for plotting: torch.Size([128, 128, 3])
Transposed matrix: torch.Size([5, 10])


> **This is vital because libraries like Matplotlib expect images in (H, W, C) format, but PyTorch uses (C, H, W).**

# Erro Scopido
## The "Contiguous" Trap
If you see the error RuntimeError: view size is not compatible with input tensor's size and stride, it's because you tried to view a tensor after transposing it.